In [1]:
# 创建clent
from openai import OpenAI
from dotenv import load_dotenv

import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=api_key,base_url="https://api.deepseek.com")

In [3]:
messages=[{"role": "user", "content": "Explain the importance of fast language models"}]

chat_completion = client.chat.completions.create(
    model="deepseek-chat",
    messages=messages,
    temperature=0,
)

chat_completion.choices[0].message.content

'Fast language models are crucial for several reasons, particularly in real-world applications where speed, efficiency, and user experience are key. Here’s why they matter:\n\n### 1. **Improved User Experience**  \n   - **Responsiveness**: Faster models reduce latency, making interactions (e.g., chatbots, virtual assistants) feel instantaneous. Users expect quick replies—delays of even a few seconds can feel frustrating.  \n   - **Scalability**: Speed enables handling more concurrent requests (e.g., in customer support or API services), improving throughput for large-scale deployments.\n\n### 2. **Cost Efficiency**  \n   - **Compute Savings**: Faster inference means lower computational costs per query, reducing cloud/GPU expenses.  \n   - **Energy Efficiency**: Optimized models consume less power, aligning with sustainability goals (important for edge devices or data centers).\n\n### 3. **Real-Time Applications**  \n   - **Live Interactions**: Speed is critical for real-time use cases 

In [4]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})
    
    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        compiletion = self.client.chat.completions.create(
            model="deepseek-chat",
            messages=self.messages,
        )
        return compiletion.choices[0].message.content

In [5]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE 

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this: 

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

In [6]:
def calculate(operation) -> float:
    return eval(operation)

def get_planet_mass(planet) -> float:
    if planet == "Earth":
        return 5.972e24
    if planet == "Mars":
        return 6.39e23
    if planet == "Jupiter":
        return 1.898e27
    if planet == "Saturn":
        return 5.683e26
    if planet == "Uranus":
        return 8.681e25
    if planet == "Neptune":
        return 1.024e26
    if planet == "Mercury":
        return 3.285e23
    if planet == "Venus":
        return 4.867e24
    return None

In [ ]:
import re

def loop(max_iterations=10, query= ""):
    tools = ["calculate", "get_planet_mass"]
    next_prompt = query
    i = 0
    
    agent = Agent(client=client, system=system_prompt)
    
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
        
        print(result)
        if "Answer:" in result:
            break
        
        chosen_tool = action[0][0]
        arg = action[0][1]
        if chosen_tool in tools:
            print(chosen_tool, arg)
            result_tool = eval(f"{chosen_tool}('{arg}')")
            next_prompt = f"Observation: {result_tool}"
        else:
            next_prompt = "Observation: Tool not found"
        
        print(next_prompt)
        continue
loop(query="What is the mass of Earth plus the mass of Saturn and all of that times 2?") 

Thought: I need to find the mass of Earth and the mass of Saturn first.
Action: get_planet_mass: Earth
PAUSE

Observation: 5.972e24
Thought: Now I need to find the mass of Saturn.
Action: get_planet_mass: Saturn
PAUSE

Observation: 5.683e26
Thought: Now I need to add the masses of Earth and Saturn, then multiply the result by 2.
Action: calculate: (5.972e24 + 5.683e26) * 2
PAUSE

Observation: 1.148544e27
Answer: The mass of Earth plus the mass of Saturn and all of that times 2 is 1.148544e27 kg.


In [9]:
import re

query = 'Action: get_planet_mass: Earth'
action = re.findall(r"Action: ([a-z_]+): (.+)", query, re.IGNORECASE)
action


[('get_planet_mass', 'Earth')]